In [2]:
# SVM
# Support vector machines (SVMs) are a set of supervised learning methods used for 
# classification, regression and outliers detection.

# The advantages of support vector machines are:
    # Effective in high dimensional spaces.
    # Still effective in cases where number of dimensions is greater than the number of samples.
    # Uses a subset of training points in the decision function (called support vectors), 
    # so it is also memory efficient.
    # Versatile: different Kernel functions can be specified for the decision function. 
    # Common kernels are provided, but it is also possible to specify custom kernels.

# The disadvantages of support vector machines include:
    # If the number of features is much greater than the number of samples, avoid over-fitting 
    # in choosing Kernel functions and regularization term is crucial.
    # SVMs do not directly provide probability estimates, these are calculated using an 
    # expensive five-fold cross-validation (see Scores and probabilities, below).

# class sklearn.svm.SVC(*, C=1.0, kernel='rbf', degree=3, gamma='scale', 
# coef0=0.0, shrinking=True, probability=False, tol=0.001, cache_size=200, 
# class_weight=None, verbose=False, max_iter=-1, 
# decision_function_shape='ovr', break_ties=False, random_state=None)

from sklearn import svm
import pandas as pd
import numpy as np

np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Define the preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values
    ('scaler', StandardScaler())  # Scale the features
])

# define and train the model
# kernals could be: ‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’.              
clf = svm.SVC(kernel="linear", C=5000, probability=True)

##############################  TRAINING  ##############################################
# data
input_data = pd.read_csv('NHANES_data_stroke_train.csv')

#under sample the non-stroke
MI_positive = input_data[input_data['stroke'] == 1]
MI_negitive = input_data[input_data['stroke'] == 2].sample(frac=.03411675511751327)
input_data = pd.concat([MI_positive, MI_negitive])

# attributes
featurenames = ["Income","Sex","Age","Race","Edu","Diastolic","Systolic","Pulse","BMI","HDL","Trig","LDL","TCHOL","kidneys_eGFR","Diabetes","CurrentSmoker","isActive","isInsured"]
X = input_data[featurenames]
y = input_data['stroke']

# impute and scale the data
X = preprocessing_pipeline.fit_transform(X)

# split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.50, random_state=42)

# train the model
clf.fit(X_train, y_train)

# print accuracy info
print("accuracy train : ", clf.score(X_train, y_train))
print("accuracy test : ", clf.score(X_test, y_test))
print("predicted probabilities:\n", clf.predict_proba(X_test))

##############################  PREDICTION  ##############################################
# load data set
new_data = pd.read_csv("DEMO.csv")

# No stroke column so get rid of it
new_data = new_data.drop(columns=['stroke'])

# get attributes
X_new = new_data[featurenames]

# impute and scale the data
X_new = preprocessing_pipeline.fit_transform(X_new)

# Make predictions on the new data, run model
new_probabilities = clf.predict_proba(X_new)[:, 0]  # for output
new_predictions = clf.predict(X_new) # unsed, just for testing ratio of MI/noMI

# Get each sample's ID and write probabilities to the output CSV
new_participant_ids = new_data['SEQN']
new_output_df = pd.DataFrame({'SEQN': new_participant_ids, 'Pred_Probability': new_probabilities})
new_output_df.to_csv('SVMpred.csv', index=False)


accuracy train :  0.7777777777777778
accuracy test :  0.6518518518518519
predicted probabilities:
 [[0.724 0.276]
 [0.483 0.517]
 [0.298 0.702]
 [0.331 0.669]
 [0.548 0.452]
 [0.350 0.650]
 [0.360 0.640]
 [0.631 0.369]
 [0.514 0.486]
 [0.387 0.613]
 [0.506 0.494]
 [0.294 0.706]
 [0.555 0.445]
 [0.592 0.408]
 [0.140 0.860]
 [0.416 0.584]
 [0.298 0.702]
 [0.828 0.172]
 [0.486 0.514]
 [0.401 0.599]
 [0.312 0.688]
 [0.477 0.523]
 [0.695 0.305]
 [0.551 0.449]
 [0.252 0.748]
 [0.824 0.176]
 [0.787 0.213]
 [0.139 0.861]
 [0.678 0.322]
 [0.544 0.456]
 [0.583 0.417]
 [0.416 0.584]
 [0.433 0.567]
 [0.619 0.381]
 [0.617 0.383]
 [0.408 0.592]
 [0.588 0.412]
 [0.488 0.512]
 [0.464 0.536]
 [0.184 0.816]
 [0.771 0.229]
 [0.424 0.576]
 [0.212 0.788]
 [0.393 0.607]
 [0.625 0.375]
 [0.421 0.579]
 [0.487 0.513]
 [0.694 0.306]
 [0.589 0.411]
 [0.555 0.445]
 [0.290 0.710]
 [0.668 0.332]
 [0.440 0.560]
 [0.232 0.768]
 [0.589 0.411]
 [0.532 0.468]
 [0.427 0.573]
 [0.466 0.534]
 [0.126 0.874]
 [0.389 0.611]
 

In [4]:
#### See which data points are critical #####
# get the support vectors
print("clf support vectors: {}".format(clf.support_vectors_))
# get indices of support vectors
print("clf support vector indices: {}".format(clf.support_))
# get number of support vectors for each class
print("clf # of support vectors in each class: {}".format(clf.n_support_))

clf support vectors: [[-1.125 -1.015 1.225 ... 0.557 0.540 -0.348]
 [0.000 0.985 -0.495 ... -1.794 0.540 -0.348]
 [-0.943 0.985 -1.068 ... 0.557 0.540 -0.348]
 ...
 [0.074 0.985 0.733 ... 0.557 0.540 -0.348]
 [-0.155 0.985 -0.086 ... -1.794 0.540 -0.348]
 [-0.903 -1.015 0.570 ... 0.557 0.540 2.882]]
clf support vector indices: [  0  13  14  15  19  22  24  32  34  37  39  40  43  46  52  53  57  61
  62  63  64  67  74  81  82  85  88  89  93  97  99 100 110 112 113 115
 123 125 132 133 134   2   3   5   6  12  16  17  31  38  41  42  45  48
  51  58  59  60  66  73  75  76  77  87  92  94 103 106 108 109 118 119
 121 122 124 127 129]
clf # of support vectors in each class: [41 36]
